# Catálogo INE

> Datos que mantiene el Instituto Nacional de Estadística de Bolivia y cuándo los actualiza

El Instituto Nacional de Estadística (INE) mantiene los datos más importantes en Bolivia. Estos valores son tomados como referencia por docenas de estudios y cientos de decisiones cada día. Sin embargo, en lugar de mantener un repositorio abierto, estructurado y versionado, el INE publica estos datos como enlaces a documentos en su nube privada. Ésto dificulta responder preguntas simples como qué datos existen y cuándo fueron actualizados. Si bien existe un calendario estadístico, el INE suele actualizar datos fuera de él. Y si bien cuentan con una [plataforma](https://anda.ine.gob.bo/) para compartir datos más correctamente, casi nunca suben datos a ella. Acá reconstruyo el catálogo de datos que mantiene el INE y publico una tabla simple donde es posible responder qué información existe y cuándo fue actualizada por última vez. Esta tabla será actualizada cada día de manera automática.

In [1]:
import pandas as pd
import datetime as dt
import pytz
from IPython import display
import locale
from itables import show, init_notebook_mode

locale.setlocale(locale.LC_TIME, "es_US.UTF8")
init_notebook_mode()

<IPython.core.display.Javascript object>

In [2]:
with open('update_time', 'r') as f:
    update_time = dt.datetime.strptime(f.read(), '%Y-%m-%d %H:%M')
catdf = pd.read_csv('catalogo_ine.csv', parse_dates=['modificado']).sort_values('modificado', ascending=False)
display.display(display.Markdown("Datos actualizados {}:".format(update_time.strftime('a las %H:%M del %-d de %B de %Y'))))

Datos actualizados a las 00:22 del 2 de febrero de 2022:

In [3]:
def draw_table(df):
    """
    Crea una tabla para explorar datos en el catálogo
    """
    colors = {
        'spreadsheet': '#f3f7ed',
        'excel': '#e1ede4',
        'pdf': '#f2e8e6',
        'rar': '#f2f0e6',
        'zip': '#f2f0e6',
        'stream': '#f7f6f5',
        'word': '#edf2f7'
    }

    dfi = df[['modificado', 'nombre', 'tipo', 'kb', 'link']].rename(columns={'modificado': 'Modificado', 'nombre': 'Nombre', 'tipo': 'Tipo', 'kb': 'Peso (KB)', 'link': '🔗'})
    dfi.Modificado = dfi.Modificado.dt.strftime('%Y-%m-%d %H:%M')
    dfi = dfi.set_index('Modificado')
    dfi['Nombre'] = dfi[['🔗', 'Nombre']].apply(lambda row: '<a style="color: #4f4f4f" href="{}"><b>{}</b></a>'.format(row[0], row[1]), axis=1)
    dfi['Tipo'] = dfi['Tipo'].apply(lambda x: '<span style="font-family:monospace; font-size:8px; border:1px solid #d7d6d6; border-radius:3px; opacity:.9; padding:3px; background:{}; color:"#000">{}</span>'.format(colors[x], x))
    dfi['Peso (KB)'] = dfi['Peso (KB)'].astype(int)
    dfi = dfi[['Nombre', 'Tipo', 'Peso (KB)']]
    show(dfi, 
         order = [], 
         scrollY="900px", 
         lengthMenu=[50,100],
         scrollCollapse=True, 
         paging=True, 
         language={
             'lengthMenu': 'Mostrar _MENU_ filas',
             'search': '&#x1F50E;&#xFE0E;', 
             'processing': 'creando tabla ...', 
             'info': '', 
             'infoEmpty': '', 
             'infoFiltered':'_TOTAL_ documentos',
             'paginate': {
                'first': "Primero",
                'previous': "Anterior",
                'next': "Siguiente",
                'last': "Último"
            },
         }, 
         columnDefs=[
             {"width": "20px", "targets": [0, 2]}, 
             {"width": "100px", "targets": 1}, 
             {'width': '5px', 'targets': [3]}
         ])

draw_table(catdf)

Loading... (need help?)
